In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from database import Database
from Filter.PB import PB
from Filter.MA import MA
from Filter.Higher_Market import Higher_Market
from Filter.DoubleRateRise import DoubleRateRise
from Filter.Chips import Chips
from Utils.GetExtraX import GetExtraX
from FeatureSelect.FeatureSelect import FeatureSelect
from FeatureSelect.FeatureXMLGenerate import FeatureXMLGenerate
from Draw.draw import Draw
from Draw.stock_data_visualizer import StockDataVisualizer
import plotly.express as px
import plotly.graph_objects as go
import time

In [ ]:
def F_Draw(filtered_data,features,*xml):
    key_list = list(features.keys())
    Draw(filtered_data,key_list,*xml)

In [ ]:
def getFliterStock(s_data,selected_companies,start_d,end_d):

    #找出符合條件的公司股價資料
    stock_fliter_data = s_data[(s_data['證券代碼'].isin(selected_companies)) & (s_data['日期'] >= start_d) & (s_data['日期'] <= end_d)]

    # 先將 DataFrame 根據 "證券代碼" 和 "日期" 進行排序
    stock_fliter_data = stock_fliter_data.sort_values(by=["證券代碼", "日期"], ascending=[True, False])

    # 計算 "區間股價變化"
    stock_fliter_data["區間股價變化"] = stock_fliter_data.groupby("證券代碼")["收盤價(元)"].transform(lambda x: x.iloc[0] - x.iloc[-1]).round(2)

    # 計算 "區間股價變化率"
    stock_fliter_data["區間股價變化率"] = stock_fliter_data.groupby("證券代碼")["收盤價(元)"].transform(lambda x: (x.iloc[-1] - x.iloc[0]) / x.iloc[-1] * 100).round(2)


    # 選擇每個分組的第一筆資料，包含 "區間股價變化" 和 "區間股價變化率" 欄位
    stock_data = stock_fliter_data.groupby("證券代碼").first().reset_index()
    return stock_data  

In [ ]:
def getFliterMarket(mk_data,start_d,end_d):
    #找出符合條件的公司指數資料
    mk_fliter_data = mk_data[(mk_data['日期'] >= start_d) & (mk_data['日期'] <= end_d)]
    # 先將 DataFrame 根據 "證券代碼" 和 "日期" 進行排序
    mk_fliter_data = mk_fliter_data.sort_values(by=["指數名稱", "日期"], ascending=[True, False])

    # 計算 "區間股價變化"
    mk_fliter_data["區間指數變化"] = mk_fliter_data.groupby("指數名稱")["指數收盤價(元)"].transform(lambda x: x.iloc[0] - x.iloc[-1]).round(2)

    # 計算 "區間股價變化率"
    mk_fliter_data["區間指數變化率"] = mk_fliter_data.groupby("指數名稱")["指數收盤價(元)"].transform(lambda x: (x.iloc[-1] - x.iloc[0]) / x.iloc[-1] * 100).round(2)

    # 選擇每個分組的第一筆資料，包含 "區間股價變化" 和 "區間股價變化率" 欄位
    market_data = mk_fliter_data.groupby("指數名稱").first().reset_index()

    return market_data 

In [ ]:
def getFliterChips(ch_data,selected_companies,start_d,end_d):
   #找出符合條件的公司籌碼資料
    ch_fliter_data = ch_data[(ch_data['證券代碼'].isin(selected_companies)) & (ch_data['日期'] >= start_d) & (ch_data['日期'] <= end_d)]
    # 先將 DataFrame 根據 "證券代碼" 和 "日期" 進行排序
    ch_fliter_data = ch_fliter_data.sort_values(by=["證券代碼", "日期"], ascending=[True, False])

    # 計算 各區間變化
    ch_fliter_data["區間董監持股變化"] = ch_fliter_data.groupby("證券代碼")["董監持股數"].transform(lambda x: x.iloc[0] - x.iloc[-1]).round(2)
    ch_fliter_data["區間融資餘額(張)變化"] = ch_fliter_data.groupby("證券代碼")["融資餘額(張)"].transform(lambda x: x.iloc[0] - x.iloc[-1]).round(2)
    ch_fliter_data["區間融券餘額(張)變化"] = ch_fliter_data.groupby("證券代碼")["融券餘額(張)"].transform(lambda x: x.iloc[0] - x.iloc[-1]).round(2)
    ch_fliter_data["區間200-400 張(比率)變化"] = ch_fliter_data.groupby("證券代碼")["200-400 張(比率)"].transform(lambda x: x.iloc[0] - x.iloc[-1]).round(2)
    ch_fliter_data["區間400-600 張(比率)變化"] = ch_fliter_data.groupby("證券代碼")["400-600 張(比率)"].transform(lambda x: x.iloc[0] - x.iloc[-1]).round(2)
    ch_fliter_data["區間600-800 張(比率)變化"] = ch_fliter_data.groupby("證券代碼")["600-800 張(比率)"].transform(lambda x: x.iloc[0] - x.iloc[-1]).round(2)
    ch_fliter_data["區間800-1000張(比率)變化"] = ch_fliter_data.groupby("證券代碼")["800-1000張(比率)"].transform(lambda x: x.iloc[0] - x.iloc[-1]).round(2)
    ch_fliter_data["區間1000張以上  (比率)變化"] = ch_fliter_data.groupby("證券代碼")["1000張以上  (比率)"].transform(lambda x: x.iloc[0] - x.iloc[-1]).round(2)


    # 定義一個函數計算區間變化率，處理分母為零的情況

    def calculate_change_rate(x):
       if ((x.iloc[-1] == 0 )&(x.iloc[0] != 0)) :
          return 10000  # 如果分母為零，設置為float64的最大值
       return ((x.iloc[-1] - x.iloc[0]) / x.iloc[-1] * 100).round(2)

    # 計算 "區間變化率"
    ch_fliter_data["區間董監持股變化率"] = ch_fliter_data.groupby("證券代碼")["董監持股數"].transform(lambda x: (x.iloc[-1] - x.iloc[0]) / x.iloc[-1] * 100).round(2)
    ch_fliter_data["區間融資餘額(張)變化率"] = ch_fliter_data.groupby("證券代碼")["融資餘額(張)"].transform(calculate_change_rate)
    ch_fliter_data["區間融券餘額(張)變化率"] = ch_fliter_data.groupby("證券代碼")["融券餘額(張)"].transform(calculate_change_rate)
    # 選擇每個分組的第一筆資料，包含 "區間股價變化" 和 "區間股價變化率" 欄位
    chips_data = ch_fliter_data.groupby("證券代碼").first().reset_index()

    return chips_data 

In [ ]:
def getFliterData(company_data,m_data,q_data,s_data,mk_data,ch_data,selected_companies,start_d,end_d):
    # 先将相关的数据转换为日期时间格式
    m_data['月報所屬日期'] = pd.to_datetime(m_data['月報所屬日期'])
    m_data['營收發布日'] = pd.to_datetime(m_data['營收發布日'])
    q_data['季報所屬日期'] = pd.to_datetime(q_data['季報所屬日期'])
    q_data['季報發布日'] = pd.to_datetime(q_data['季報發布日'])
    chips_data= getFliterChips(ch_data,selected_companies,start_d,end_d)
    market_data =getFliterMarket(mk_data,start_d,end_d)
    stock_data =getFliterStock(s_data,selected_companies,start_d,end_d)
    stock_data['日期'] = pd.to_datetime(stock_data['日期'])
    # 对 m_data 按照 '營收發布日' 和 q_data 按照 '季報發布日' 从新到旧排序
    m_data = m_data.sort_values(by='營收發布日', ascending=False)
    q_data = q_data.sort_values(by='季報發布日', ascending=False)

    # 定义一个通用的函数来查找 '發布日' 对应的 '所屬日期'
    def find_next_date(row, data, date_column, target_date_column):
        company_id = int(row['證券代碼'])
        date = row['日期']
        
        # 从 data 中筛选出相同 '證券代碼' 的记录
        correspond_company_data = data[data['證券代碼'] == company_id]
        
        # 用 iterrows() 迭代行
        for _, m_row in correspond_company_data.iterrows():
            if date > m_row[date_column]:
                return m_row[target_date_column]
        
        # 如果没有找到匹配的记录，可以返回一个默认值，或者根据你的需求做相应处理
        return None

    # 将 '月報所屬日期' 列填充为对应的 '營收所屬日期'
    stock_data['月報所屬日期'] = stock_data.apply(find_next_date, args=(m_data, '營收發布日', '月報所屬日期',), axis=1)

    # 将 '季報所屬日期' 列填充为对应的 '季報所屬日期'
    stock_data['季報所屬日期'] = stock_data.apply(find_next_date, args=(q_data, '季報發布日', '季報所屬日期',), axis=1)
    final_merged_df = pd.merge(stock_data, m_data, on=['月報所屬日期', '證券代碼'], how='left').merge(q_data, on=['季報所屬日期', '證券代碼'], how='left').merge(chips_data, on=['日期', '證券代碼'], how='left').merge(company_data, on=['證券代碼'], how='left').merge(market_data,left_on=['日期', '上市別'], right_on=['日期', '指數名稱'], how='left')
    return final_merged_df

In [ ]:
def analysis(func_name,start_d,end_d,*xml):

    # 讀取資料
    db = Database()
    # #TODO:一次取完變成data?
    # stock_data = db.get_stock(start_d,end_d).drop('id', axis=1)
    company_data =db.get_company_basic()
    q_data=db.get_finance_report_q()
    m_data=db.get_finance_report_m()
    s_data = db.get_stock(start_d,end_d)
    mk_data = db.get_market_index(start_d,end_d)
    ch_data = db.get_chips(start_d,end_d)

    #選擇策略+防呆
    if func_name == "F_PB":
        selected_companies = PB(q_data)

    elif func_name == "F_DoubleRateRise":
        selected_companies = DoubleRateRise(q_data,start_d)
        
    elif func_name == "F_Chips":
        selected_companies = Chips(ch_data,*xml)

    elif func_name == "F_MA":
        selected_companies = MA(s_data)
        
    elif func_name == "F_Higher_Market":
        selected_companies = Higher_Market(s_data,company_data,mk_data,start_d,end_d)
    else:
        return "請選擇合適篩選標準"
    filtered_data=getFliterData(company_data,m_data,q_data,s_data,mk_data,ch_data,selected_companies,start_d,end_d)
    
    print(f"符合條件股數:{len(filtered_data)}")

    visualizer = StockDataVisualizer(filtered_data)
    visualizer.get_heat_map()
    visualizer.get_industry_pie()
    visualizer.get_bar_chart()
   
    if xml:
        addX_data = GetExtraX(filtered_data,xml[0])
    else:
        addX_data = filtered_data

    features = FeatureSelect(addX_data)

    FeatureXMLGenerate(features)

    return features, filtered_data
    

In [ ]:

features, filtered_data = analysis("F_ALL",'2023/03/15','2023/09/15','XML_chip1031.xml')

In [ ]:
# F_Draw(filtered_data,features,'v1TestXML1030.xml')

In [ ]:
F_Draw(filtered_data,features)